In [1]:
import openai
import yaml
with open("conf/index.yaml") as f:
    credentials = yaml.safe_load(f)
openai.api_key = credentials['environment_variables']['OPENAI_API_KEY']
model_name = "gpt-3.5-turbo"
system_prompt = ""
user_query = ""

In [2]:
import pandas as pd
df = pd.read_csv('./data/ground_truth.csv')
print(df.head(10))
# df = df.head(100)
arguments = list(df['argument'].values)
target_KPs = list(df['key_point'].values)
print(arguments)
print(len(arguments))

                                           topic  \
0  Assisted suicide should be a criminal offence   
1  Assisted suicide should be a criminal offence   
2  Assisted suicide should be a criminal offence   
3  Assisted suicide should be a criminal offence   
4  Assisted suicide should be a criminal offence   
5  Assisted suicide should be a criminal offence   
6  Assisted suicide should be a criminal offence   
7  Assisted suicide should be a criminal offence   
8  Assisted suicide should be a criminal offence   
9  Assisted suicide should be a criminal offence   

                                            argument  \
0  a cure or treatment may be discovered shortly ...   
1  A patient should be able to decide when they h...   
2  a person has the right to end their suffering ...   
3  a person should have the dignity to choose how...   
4  a person should have the right to be able to c...   
5  a person should have the right to die on their...   
6  a person should not be forced in

In [3]:
def make_api_request(a):
    rep = openai.ChatCompletion.create(
        # request_timeout = 10,
        model = model_name,
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": a}
        ]
    )

    re = rep["choices"][0]["message"]["content"]
    return re


In [4]:
predict_KPs = []
system_prompt = \
        """
        Generate the key point of the given argument. The output format is like this:
        Key Point: ...
        """
for argument in arguments[:1]:
    result = make_api_request(argument)
    print(len(predict_KPs))
    print(result)
    predict_KPs.append(result)


0
Key Point: Prematurely ending someone's life can prevent them from benefiting from potential cures or treatments that could be discovered in the future.


In [17]:
print(len(predict_KPs))

1093


In [ ]:
print(target_KPs)

In [ ]:
print(len(target_KPs))
print(len(predict_KPs))

### CSV file

In [ ]:
print(len(arguments))
print(len(target_KPs))
print(len(predict_KPs))

In [26]:
import pandas as pd

# Your three lists


# Create a DataFrame
df = pd.DataFrame({'arguments': arguments, 'target_KP': target_KPs, 'predict_KP': predict_KPs})

# Specify the CSV file path
csv_file_path = 'gpt-3.5-turbo.csv'

# Save the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)

print(f'The DataFrame has been written to {csv_file_path}')



The DataFrame has been written to gpt-3.5-turbo.csv


### Read result from csv file

In [14]:
import pandas as pd

# Specify the CSV file path
csv_file_path = 'result/gpt-3.5-turbo.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Extract each column as a separate list

predict = df['predict_KP'].tolist()
target = df['target_KP'].tolist()

# Print or use the lists as needed
print("Predict_KPs:", predict)

Predict_KPs: ["In some cases, a potential cure or treatment may be discovered shortly after a person's unnecessary death.", 'Patients should have the autonomy to determine when they are satisfied with the level of care they receive.', 'Individuals should have the right to end their suffering, and those who choose to offer assistance should not be penalized.', 'Every individual should have the right to autonomously decide the manner in which they end their own life, as it is a matter of personal dignity and choice.', 'Individuals should have the right to choose whether they want to live or die.', 'Individuals should have the autonomy and freedom to make decisions about their own lives, including the decision to die on their own terms.', 'Individuals should have the right to choose their own end-of-life options to avoid unnecessary suffering.', 'Allowing assisted suicide could potentially open the door for greedy relatives to exploit vulnerable family members, using their illness as a me

### BLEU Score

In [15]:
import sacrebleu

# Example sentences
candidates = predict
references = target

# Join sentences into strings
candidate_str = "\n".join(candidates)
reference_str = "\n".join(references)

# Calculate BLEU score
bleu = sacrebleu.raw_corpus_bleu(candidate_str, [reference_str])
print("BLEU Score:", bleu.score)

BLEU Score: 5.429924949511072


### BERT Score

In [8]:
from evaluate import load
bertscore = load("bertscore")
result = bertscore.compute(predictions=predict, references=target, lang="en")['f1']
print(result)
print(sum(result)/len(result))


[0.8630496263504028, 0.9128193855285645, 0.8513220548629761, 0.9217226505279541, 0.948319673538208, 0.9318950772285461, 0.852904736995697, 0.8817904591560364, 0.8862724304199219, 0.8643102645874023, 0.852054238319397, 0.8837476968765259, 0.9057462811470032, 0.8588156700134277, 0.9000350832939148, 0.8636276125907898, 0.9014069437980652, 0.9195679426193237, 0.9021428227424622, 0.897433340549469, 0.8973255753517151, 0.9041566252708435, 0.8838711380958557, 0.8993544578552246, 0.8748055696487427, 0.9068295955657959, 0.9038687348365784, 0.9129427671432495, 0.8719207644462585, 0.8997296690940857, 0.8646517992019653, 0.9097696542739868, 0.8736065030097961, 0.8777973651885986, 0.9303032159805298, 0.9099940657615662, 0.9099153876304626, 0.9061897397041321, 0.9203962683677673, 0.8903139233589172, 0.9354386925697327, 0.8885809183120728, 0.8899160027503967, 0.8846173286437988, 0.8800337314605713, 0.9499606490135193, 0.8887707591056824, 0.8999165296554565, 0.8893195390701294, 0.887683629989624, 0.89

### Export Result

In [13]:
f = open("./result/gpt-3.5-turbo.txt",'w', encoding='UTF-8')
for batch_id, kp in enumerate(target_KPs):
    print(batch_id)
    print("-------------------Processing-------------------")
    f.write(str(batch_id)+"\n")
    f.write("argument:\n")
    f.write(arguments[batch_id]+"\n")
    f.write("target:\n")
    f.write(target[batch_id]+"\n")
    f.write("predict:\n")
    f.write(predict[batch_id]+"\n")
    f.write("\n")

print("-----------------Finished--------------------------")

0
-------------------Processing-------------------
1
-------------------Processing-------------------
2
-------------------Processing-------------------
3
-------------------Processing-------------------
4
-------------------Processing-------------------
5
-------------------Processing-------------------
6
-------------------Processing-------------------
7
-------------------Processing-------------------
8
-------------------Processing-------------------
9
-------------------Processing-------------------
10
-------------------Processing-------------------
11
-------------------Processing-------------------
12
-------------------Processing-------------------
13
-------------------Processing-------------------
14
-------------------Processing-------------------
15
-------------------Processing-------------------
16
-------------------Processing-------------------
17
-------------------Processing-------------------
18
-------------------Processing-------------------
19
-------------------